In [255]:
import pandas as pd
import numpy as np
import tensorflow as tf

SHUFF_BUFFER = 500
BATCH_SIZE = 2

In [256]:
df = pd.read_csv('../datasets/creditcard.csv')
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [257]:
df.shape

(284807, 31)

In [258]:
from sklearn.model_selection import train_test_split

In [328]:
# Balance the data
fraud = df[df.Class == 1]
not_fraud = df[df.Class==0]
not_fraud = not_fraud.sample(n=fraud.shape[0])
df = pd.concat([fraud,not_fraud], axis=0)
df.Class.value_counts()

1    492
0    492
Name: Class, dtype: int64

In [329]:
X = df.iloc[:, :-1]
X.Amount = (X.Amount - X.Amount.mean())/X.Amount.std()
y = df.iloc[:, -1]

In [360]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=df.Class)

In [361]:
# 0 is non-fraud 1 is fraud
y.value_counts()

1    492
0    492
Name: Class, dtype: int64

In [362]:
X_train.shape

(885, 30)

In [363]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [364]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [365]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [368]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(X_train[0].shape),
        tf.keras.layers.Conv1D(256,5, activation='relu'),
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(128,5, activation='relu'),
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [369]:
EPOCHS = 200
model = get_model()
model.fit(x=X_train, y=y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1)

Epoch 1/200
398/398 [==============================] - 2s 3ms/step - loss: 0.5660 - accuracy: 0.8410 - val_loss: 0.2736 - val_accuracy: 0.9101
Epoch 2/200
398/398 [==============================] - 1s 3ms/step - loss: 0.2514 - accuracy: 0.9035 - val_loss: 0.1810 - val_accuracy: 0.9551
Epoch 3/200
398/398 [==============================] - 1s 2ms/step - loss: 0.2105 - accuracy: 0.9252 - val_loss: 0.1981 - val_accuracy: 0.9438
Epoch 4/200
398/398 [==============================] - 1s 3ms/step - loss: 0.1831 - accuracy: 0.9361 - val_loss: 0.1634 - val_accuracy: 0.9438
Epoch 5/200
398/398 [==============================] - 1s 3ms/step - loss: 0.2154 - accuracy: 0.9011 - val_loss: 0.2280 - val_accuracy: 0.8764
Epoch 6/200
398/398 [==============================] - 1s 2ms/step - loss: 0.1604 - accuracy: 0.9327 - val_loss: 0.1288 - val_accuracy: 0.9663
Epoch 7/200
398/398 [==============================] - 1s 3ms/step - loss: 0.2093 - accuracy: 0.9237 - val_loss: 0.1628 - val_accuracy: 0.9438

Epoch 58/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0841 - accuracy: 0.9756 - val_loss: 0.1703 - val_accuracy: 0.9663
Epoch 59/200
398/398 [==============================] - 1s 3ms/step - loss: 0.1211 - accuracy: 0.9465 - val_loss: 0.1343 - val_accuracy: 0.9551
Epoch 60/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0347 - accuracy: 0.9917 - val_loss: 0.1354 - val_accuracy: 0.9663
Epoch 61/200
398/398 [==============================] - 1s 2ms/step - loss: 0.0431 - accuracy: 0.9893 - val_loss: 0.2548 - val_accuracy: 0.9438
Epoch 62/200
398/398 [==============================] - 1s 2ms/step - loss: 0.1062 - accuracy: 0.9710 - val_loss: 0.1270 - val_accuracy: 0.9663
Epoch 63/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0446 - accuracy: 0.9868 - val_loss: 0.1582 - val_accuracy: 0.9551
Epoch 64/200
398/398 [==============================] - 1s 3ms/step - loss: 0.1209 - accuracy: 0.9643 - val_loss: 0.1500 - val_accuracy:

Epoch 115/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0675 - accuracy: 0.9810 - val_loss: 0.3157 - val_accuracy: 0.9551
Epoch 116/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0706 - accuracy: 0.9801 - val_loss: 0.2337 - val_accuracy: 0.9663
Epoch 117/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0188 - accuracy: 0.9958 - val_loss: 0.2123 - val_accuracy: 0.9551
Epoch 118/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0545 - accuracy: 0.9839 - val_loss: 0.2523 - val_accuracy: 0.9438
Epoch 119/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0759 - accuracy: 0.9734 - val_loss: 0.2463 - val_accuracy: 0.9438
Epoch 120/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0197 - accuracy: 0.9908 - val_loss: 0.2365 - val_accuracy: 0.9551
Epoch 121/200
398/398 [==============================] - 2s 5ms/step - loss: 0.0206 - accuracy: 0.9930 - val_loss: 0.2602 - val_ac

398/398 [==============================] - 1s 3ms/step - loss: 0.0106 - accuracy: 0.9963 - val_loss: 0.2728 - val_accuracy: 0.9438
Epoch 172/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0349 - accuracy: 0.9901 - val_loss: 0.2370 - val_accuracy: 0.9326
Epoch 173/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0395 - accuracy: 0.9868 - val_loss: 0.1963 - val_accuracy: 0.9326
Epoch 174/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0100 - accuracy: 0.9977 - val_loss: 0.2117 - val_accuracy: 0.9438
Epoch 175/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0220 - accuracy: 0.9933 - val_loss: 0.2035 - val_accuracy: 0.9438
Epoch 176/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0236 - accuracy: 0.9906 - val_loss: 0.2509 - val_accuracy: 0.9326
Epoch 177/200
398/398 [==============================] - 1s 3ms/step - loss: 0.0140 - accuracy: 0.9898 - val_loss: 0.3394 - val_accuracy: 0.9326

In [370]:
model.save_weights('./weights/naive-nn')

In [371]:
loss, acc = model.evaluate(tf.convert_to_tensor(X_test), y_test, verbose=2)
print(f" loss {loss} accuracy {acc*100}")

4/4 - 0s - loss: 0.2713 - accuracy: 0.9293
 loss 0.2713235914707184 accuracy 92.92929172515869


In [372]:
preds= model.predict(X_test)

In [373]:
preds.round()

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],

In [374]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds.round())

0.9292929292929293

In [375]:
len(preds)

99

In [376]:
len(y_test[y_test ==1])

49

In [377]:
print(len(np.round(preds)[np.round(preds)==1]))
print(len(np.round(preds)[np.round(preds)==0]))

44
55


In [378]:
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, preds[:,0].round())

array([[49,  1],
       [ 6, 43]])